<div class='alert alert-warning'>

NumPy's interactive examples are experimental and may not always work as expected, with high load times especially on low-resource platforms, , and the version of NumPy might not be in sync with the one you are browsing the documentation for. If you encounter any issues, please report them on the [NumPy issue tracker](https://github.com/numpy/numpy/issues).

</div>

In [ ]:
import numpy as np
from io import StringIO   # StringIO behaves like a file object
c = StringIO("0 1\n2 3")
np.loadtxt(c)

array([[0., 1.],
       [2., 3.]])

In [ ]:
d = StringIO("M 21 72\nF 35 58")
np.loadtxt(d, dtype={'names': ('gender', 'age', 'weight'),
                     'formats': ('S1', 'i4', 'f4')})

array([(b'M', 21, 72.), (b'F', 35, 58.)],
      dtype=[('gender', 'S1'), ('age', '<i4'), ('weight', '<f4')])

In [ ]:
c = StringIO("1,0,2\n3,0,4")
x, y = np.loadtxt(c, delimiter=',', usecols=(0, 2), unpack=True)
x

array([1., 3.])

In [ ]:
y

array([2., 4.])

The `converters` argument is used to specify functions to preprocess the
text prior to parsing. `converters` can be a dictionary that maps
preprocessing functions to each column:


In [ ]:
s = StringIO("1.618, 2.296\n3.141, 4.669\n")
conv = {
    0: lambda x: np.floor(float(x)),  # conversion fn for column 0
    1: lambda x: np.ceil(float(x)),  # conversion fn for column 1
}
np.loadtxt(s, delimiter=",", converters=conv)

array([[1., 3.],
       [3., 5.]])

`converters` can be a callable instead of a dictionary, in which case it
is applied to all columns:


In [ ]:
s = StringIO("0xDE 0xAD\n0xC0 0xDE")
import functools
conv = functools.partial(int, base=16)
np.loadtxt(s, converters=conv)

array([[222., 173.],
       [192., 222.]])

This example shows how `converters` can be used to convert a field
with a trailing minus sign into a negative number.


In [ ]:
s = StringIO("10.01 31.25-\n19.22 64.31\n17.57- 63.94")
def conv(fld):
    return -float(fld[:-1]) if fld.endswith("-") else float(fld)

np.loadtxt(s, converters=conv)

array([[ 10.01, -31.25],
       [ 19.22,  64.31],
       [-17.57,  63.94]])

Using a callable as the converter can be particularly useful for handling
values with different formatting, e.g. floats with underscores:


In [ ]:
s = StringIO("1 2.7 100_000")
np.loadtxt(s, converters=float)

array([1.e+00, 2.7e+00, 1.e+05])

This idea can be extended to automatically handle values specified in
many different formats, such as hex values:


In [ ]:
def conv(val):
    try:
        return float(val)
    except ValueError:
        return float.fromhex(val)
s = StringIO("1, 2.5, 3_000, 0b4, 0x1.4000000000000p+2")
np.loadtxt(s, delimiter=",", converters=conv)

array([1.0e+00, 2.5e+00, 3.0e+03, 1.8e+02, 5.0e+00])

Or a format where the ``-`` sign comes after the number:


In [ ]:
s = StringIO("10.01 31.25-\n19.22 64.31\n17.57- 63.94")
conv = lambda x: -float(x[:-1]) if x.endswith("-") else float(x)
np.loadtxt(s, converters=conv)

array([[ 10.01, -31.25],
       [ 19.22,  64.31],
       [-17.57,  63.94]])

Support for quoted fields is enabled with the `quotechar` parameter.
Comment and delimiter characters are ignored when they appear within a
quoted item delineated by `quotechar`:


In [ ]:
s = StringIO('"alpha, #42", 10.0\n"beta, #64", 2.0\n')
dtype = np.dtype([("label", "U12"), ("value", float)])
np.loadtxt(s, dtype=dtype, delimiter=",", quotechar='"')

array([('alpha, #42', 10.), ('beta, #64',  2.)],
      dtype=[('label', '<U12'), ('value', '<f8')])

Quoted fields can be separated by multiple whitespace characters:


In [ ]:
s = StringIO('"alpha, #42"       10.0\n"beta, #64" 2.0\n')
dtype = np.dtype([("label", "U12"), ("value", float)])
np.loadtxt(s, dtype=dtype, delimiter=None, quotechar='"')

array([('alpha, #42', 10.), ('beta, #64',  2.)],
      dtype=[('label', '<U12'), ('value', '<f8')])

Two consecutive quote characters within a quoted field are treated as a
single escaped character:


In [ ]:
s = StringIO('"Hello, my name is ""Monty""!"')
np.loadtxt(s, dtype="U", delimiter=",", quotechar='"')

array('Hello, my name is "Monty"!', dtype='<U26')

Read subset of columns when all rows do not contain equal number of values:


In [ ]:
d = StringIO("1 2\n2 4\n3 9 12\n4 16 20")
np.loadtxt(d, usecols=(0, 1))

array([[ 1.,  2.],
       [ 2.,  4.],
       [ 3.,  9.],
       [ 4., 16.]])